<a href="https://colab.research.google.com/github/yang-su2000/Authorship-Identification-with-NLP/blob/main/run_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("drive/MyDrive/Colab Notebooks/Authorship Identification/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load C50 Dataset

In [ ]:
from os import listdir
from os.path import isfile, isdir, join
import numpy as np

author_names = [name for name in listdir('C50/C50train/') if isdir(join('C50/C50train/', name))]
print('There are', len(author_names), 'authors (both train and test), the first 3:', author_names[:3])

train_article_names = [[article_name for article_name in listdir('C50/C50train/' + author_name + '/') if isfile(join('C50/C50train/' + author_name + '/', article_name))]
                  for author_name in author_names]
print('There are', len(train_article_names[0]), '(train) articles written by the first author')
test_article_names = [[article_name for article_name in listdir('C50/C50test/' + author_name + '/') if isfile(join('C50/C50test/' + author_name + '/', article_name))]
                  for author_name in author_names]
print('There are', len(test_article_names[0]), '(test) articles written by the first author')

train_articles = np.empty((2500, 2), dtype=object)
for i, author_name in enumerate(author_names):
  for j, article_name in enumerate(train_article_names[i]):
    file_path = 'C50/C50train/' + author_name + '/' + article_name
    with open(file_path, 'r') as f:
      train_articles[i * len(author_names) + j, 0] = author_name
      train_articles[i * len(author_names) + j, 1] = f.read()
print('The shape of train articles is', train_articles.shape)

test_articles = np.empty((2500, 2), dtype=object)
for i, author_name in enumerate(author_names):
  for j, article_name in enumerate(test_article_names[i]):
    file_path = 'C50/C50test/' + author_name + '/' + article_name
    with open(file_path, 'r') as f:
      test_articles[i * len(author_names) + j, 0] = author_name
      test_articles[i * len(author_names) + j, 1] = f.read()
print('The shape of test articles is', test_articles.shape)

In [ ]:
print('The average length of text of an article in training set is', np.mean([len(content.split()) for content in train_articles[:,1]]), 
      'and in test set is', np.mean([len(content.split()) for content in test_articles[:,1]]))

The average length of text of an article in training set is 501.77 and in test set is 509.5508


In [ ]:
import pandas as pd

train_df = pd.DataFrame(data=train_articles, columns=['author','content'])
train_df.to_csv('C50/train.csv', index=False)
test_df = pd.DataFrame(data=test_articles, columns=['author','content'])
test_df.to_csv('C50/test.csv', index=False)

### Load C50 Dataset (if already loaded)

In [ ]:
import pandas as pd

train_df = pd.read_csv('C50/train.csv')
test_df = pd.read_csv('C50/test.csv')

In [ ]:
train_df.head()

,author,content
0,BernardHickey,Westpac Banking Corp Ltd is expected to report...
1,BernardHickey,"Australia's second-largest bank, Westpac Banki..."
2,BernardHickey,Airline giant British Airways Plc (BA) is unli...
3,BernardHickey,An official inquiry into Australia's financial...
4,BernardHickey,World yeast and spices giant Burns Philp &amp;...


### Load "All the News" 1 Dataset

In [ ]:
import pandas as pd

random_seed = 42
train_df = pd.read_csv('All the News/articles1.csv')
train_df = train_df[(train_df.publication == 'Breitbart')]
top10_authors = train_df[['author','id']].groupby('author').count().sort_values(by='id', ascending=False).head(10).reset_index() # this == what the paper proposed
print('The authors are', top10_authors.author.values)
train_df = pd.merge(train_df, top10_authors[['author']], on='author', how='right')
train_df = train_df.groupby(by='author').sample(500, random_state=random_seed)
train_df.shape

The authors are ['Breitbart News' 'Pam Key' 'Charlie Spiering' 'Jerome Hudson'
 'John Hayward' 'Daniel Nussbaum' 'AWR Hawkins' 'Ian Hanchett'
 'Joel B. Pollak' 'Alex Swoyer']


(5000, 10)

### Preprocessing

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 25.2 MB/s 
     |████████████████████████████████| 6.6 MB 47.6 MB/s 
     |████████████████████████████████| 163 kB 70.0 MB/s 


In [ ]:
import torch
import numpy as np
from sklearn import preprocessing
from transformers import BertTokenizer

le = preprocessing.LabelEncoder()
le.fit(train_df['author'])
random_seed = 42
test_frac = 0.2
test_df = train_df.sample(frac=test_frac, random_state=random_seed)
train_df = train_df.drop(test_df.index)
train_df['author_id'] = le.transform(train_df['author']).astype(int)
test_df['author_id'] = le.transform(test_df['author']).astype(int)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
test_df.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content,author_id
3762,31449,50215,Still Climbing: Donald Trump Reaches New High ...,Breitbart,Charlie Spiering,2016-09-19,2016.0,9.0,NaN,Republican presidential nominee Donald Trump i...,3
7125,20247,38993,Juan Williams: Members of Media ’Disappointed’...,Breitbart,Ian Hanchett,2016-12-05,2016.0,12.0,NaN,"On Monday’s broadcast of “The Five,” Fox News ...",5
6792,8207,26943,"Cruz: Gorsuch A Principled Constitutionalist, ...",Breitbart,Ian Hanchett,2017-01-31,2017.0,1.0,NaN,"During an interview on CNN on Tuesday, Senator...",5
1364,29226,47990,Politico: Hillary Clinton a ’Broccoli Candidat...,Breitbart,Breitbart News,2016-01-08,2016.0,1.0,NaN,"Glenn Thrush, a solid progressive, writes at P...",2
8102,16640,35377,Donald Trump: Hillary Clinton’s ’Basket of Dep...,Breitbart,Alex Swoyer,2016-09-10,2016.0,9.0,NaN,Donald Trump is firing back after Hillary Clin...,1


In [ ]:
print('The average length of text of an article in training set is', np.mean([len(content.split()) for content in train_df['content'].values]), 
      'and in test set is', np.mean([len(content.split()) for content in test_df['content'].values]))

The average length of text of an article in training set is 421.358 and in test set is 445.253


In [ ]:
random_seed = 42
valid_frac = 0.1 / (1 - test_frac)
valid_df = train_df.sample(frac=valid_frac, random_state=random_seed)
train_df = train_df.drop(valid_df.index)

### Dataset Class

In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = df['author_id'].values
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['content']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

### Model Class

In [ ]:
from torch import nn
from transformers import BertModel, BertForSequenceClassification, get_linear_schedule_with_warmup

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super().__init__()

        self.bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=50)
        # self.dropout = nn.Dropout(dropout)
        # self.linear = nn.Linear(768, 50)
        # self.relu = nn.ReLU()

    def forward(self, input_id, labels):

        # pooled_output, = self.bert(input_ids=input_id, attention_mask=mask, return_dict=False)
        # pooled_output = pooled_output[:,0,:]
        # dropout_output = self.dropout(pooled_output)
        # linear_output = self.linear(dropout_output)
        # final_layer = self.relu(linear_output)
        # print(input_id.shape, labels.shape)
        final_layer = self.bert(input_id, labels=labels)

        return final_layer

### Training

In [ ]:
!git clone https://gist.github.com/NTT123/4596e5533e573c8ceab2f319ab5d36a2 jslog
import random
import math
import time
from jslog.jslogger import JSLogger

logger = JSLogger('train/valid loss', ['train', 'valid'])
logger_ = JSLogger('train/valid accuracy', ['train', 'valid'])

fatal: destination path 'jslog' already exists and is not an empty directory.


In [ ]:
from torch.optim import AdamW
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=8, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=8)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=100,
        num_training_steps=len(train_dataloader) * epochs,
    )

    if use_cuda:

            print('cuda version', torch.__version__)
            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0
            model.train()

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                # mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, train_label)
                
                # batch_loss = criterion(output, train_label.long())
                batch_loss = output.loss
                total_loss_train += batch_loss.item()
                
                acc = (output.logits.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
                lr_scheduler.step()
            
            total_acc_val = 0
            total_loss_val = 0
            model.eval()

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, val_label)

                    # batch_loss = criterion(output, val_label.long())
                    batch_loss = output.loss
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.logits.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .4f} \
                | Train Accuracy: {total_acc_train / len(train_data): .4f} \
                | Val Loss: {total_loss_val / len(val_data): .4f} \
                | Val Accuracy: {total_acc_val / len(val_data): .4f}')
            
            logger.log(epoch_num + 1, {'train': total_loss_train / len(train_data), 'valid': total_loss_val / len(val_data)})
            logger_.log(epoch_num + 1, {'train': total_acc_train / len(train_data), 'valid': total_acc_val / len(val_data)})

logger.show()
logger_.show()             
EPOCHS = 2
model = BertClassifier()
print('There are', sum(p.numel() for p in model.parameters()), 'parameters')
LR = 2e-5
              
train(model, train_df, valid_df, LR, EPOCHS)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

There are 109520690 parameters
cuda version 1.12.1+cu113


100%|██████████| 438/438 [05:36<00:00,  1.30it/s]


Epochs: 1 | Train Loss:  0.2475                 | Train Accuracy:  0.5034                 | Val Loss:  0.0963                 | Val Accuracy:  0.8520


...

<IPython.core.display.Javascript object>

100%|██████████| 438/438 [05:53<00:00,  1.24it/s]


Epochs: 2 | Train Loss:  0.0681                 | Train Accuracy:  0.8894                 | Val Loss:  0.0538                 | Val Accuracy:  0.9040


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Evaluate

In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=32)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    model.eval()
    
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              # mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, test_label)

              acc = (output.logits.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .4f}')
    
evaluate(model, test_df)

Test Accuracy:  0.8710
